# U2 - a lab activity on search strategies
---

## Prerequisites

- Python 3 (correctly installed)

- module `numpy`

- the AIMA libraries for search; two alternatives:
    - either (git-)clone the AIMA repository: <https://github.com/aimacode/aima-python>
    - or download from the course site the `search.py` file

- file `reporting.py`: contains some useful functions for inspecting what's going on

- file `utils.py`: it is used by some search implementations, simply place it in the same folder.

---


## The AIMA search library

The library provided with the textbook contains an implementation of the most notable search strategies.

To use them, two main steps:
1. **Decide a representation of a state**. A state can be represented with any object you like. However, in order to correctly use the Graph-search algorithms, the `__eq__()` method should be overrided correctly. *Suggestion: in the first exercise, opt to represent the state using a tuple. The `__eq__()` method iis already properly overridden, so it would be an easy way for starting with.*

2. **Define a new class**, that will inherit form the class `Problem`, and that will contain the methods and the attributes needed to apply the search strategies. In particular, our new subclass of `Problem` will have to implement/override:
    - constructor `def __init__(self, initial, goal=None)`:
        
        > *'''The constructor specifies the initial state, and possibly a goal state, if there is a unique goal. Your subclass's constructor can add other arguments.'''*
    
    - method `def actions(self, state):`
        
        > *'''Return the actions that can be executed in the given state. The result would typically be a list, but if there are many actions, consider yielding them one at a time in an iterator, rather than building them all at once.'''*
        
    - method `def result(self, state, action):`
        
        > *'''Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state).'''*

    - method `def goal_test(self, state):`
        
        > *'''Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough.'''*
        
    - method `def path_cost(self, c, state1, action, state2):`
        
        > *'''Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2. If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1.'''*
        
    - method `def h(self, node):`
		
        > *'''Returns the heuristic applied to the Node node'''*

## The U2 Problem

*(This problem was given as an exam the 16th of December 2005)*  
U2 are giving a concert in Dublin. **There are still 17 minutes.** Unfortunately, to reach the stage, the members of the band must cross a small, dark, and dangerous bridge... do not despair! They have a torch!!! (only one).

The bridge allows the passing of two persons at a time. The torch is mandatory to cross the bridge, and should be brought back and forth (cannot be thrown...). All the members are on the wrong side of the bridge, far from the stage...

Every member of the U2 walks at a different velocity, and they takes different time to cross the bridge: 
- Bono, **1 minute** 
- Edge, **2 minutes**
- Adam, **5 minutes**
- Larry, **10 minutes**
  
If two members cross the bridge together, it will take them the highest time to cross it (i.e., the faster will walk at the velocity of the slower).  
For example: if Bono and Larry will cross together, it will take them *10 minutes* to get over the bridge. If Larry brings back the torch, another 10 minutes will pass, and the mission will be failed.

**Suggestion**: To limit the search space, suppose that the members move always in couple in one direction, and only one member brings back the torch.

---

### About the heuristic

> - At most, only two persons can move at a time.
> - Let us group the members in couples.
> - Sort the member in descending order on the base of the crossing time, and:
>> - put in the first group the two slower members
>> - put in the second group the remaining members
>> - Every group will move at the velocity of the slowest in the group.
> - The heuristic function is the sum of the time required by each group still in the wrong side of the bridge.
> - Is this heuristic function admissible? Will it find the best solution?

---

### About the implementation

- Which state representation?

- Which actions?

- Which is the goal?

- Which path cost?


In [10]:
from search import *
from reporting import *


class U2(Problem):
    """Scelgo come rappresentazione dello stato una tupla di tuple: la prima contenente i nomi dei membri ancora sulla sponda di partenza,
    la seconda con i membri sull'altra sponda, poi un intero che indica la presenza (1) o meno (0) della torcia sulla sponda iniziale"""

    ctime = { #crossing time of the members
        "Bono": 1,
        "Edge": 2,
        "Adam": 5,
        "Larry": 10
    }

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        # prendo gli stati iniziale e goal passati e li metto in ordine decrescente di tempo:
        initial = (tuple(sorted(initial[0], reverse=True, key=lambda x:self.ctime[x])),  
                        tuple(sorted(initial[1], reverse=True, key=lambda x:self.ctime[x])),
                        initial[2])
        goal = (tuple(sorted(goal[0], reverse=True, key=lambda x:ctime[x])),
                        tuple(sorted(goal[1], reverse=True, key=lambda x:self.ctime[x])),
                        goal[2])
        # remember to invoke the constructor of the parent 
        super().__init__(initial, goal)
    
    # TO BE IMPLEMENTED
    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        if state[2]:
            sponda=state[0]
        else:
            sponda=state[1]
        validMoves = []

        # Crea le coppie di membri che possono attraversare insieme (prima, seconda) con il tempo di attraversamento maggiore
        validMoves += [(first, second) for first in sponda
                   for second in sponda if first != second and self.ctime[first] > self.ctime[second]]
    
        # Aggiungi anche i membri singoli che possono attraversare da soli
        validMoves += [member for member in sponda]
        return validMoves

    # TO BE IMPLEMENTED
    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        # Determino la sponda di partenza e di destinazione in base alla posizione della torcia
        source, dest = (list(state[0]), list(state[1])) if state[2] else (list(state[1]), list(state[0]))

        if len(action) == 2:
            for x in action:
                source.remove(x)
                dest.append(x)
        else:
            source.remove(action)
            dest.append(action)

        source.sort(reverse=True, key=lambda x:self.ctime[x])
        dest.sort(reverse=True, key=lambda x:self.ctime[x])

        if state[2]:
            return (tuple(source), tuple(dest), 0)
        else:
            return (tuple(dest), tuple(source), 1)


    # if it is fine, leave it as it is
    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    # IMPLEMENT ONLY IF you want to experiment with uniform cost and informed strategies
    # metto come costo il tempo di percorrenza
    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2. If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        if len(action) == 2:
            return c + self.ctime[action[0]]
        else:
            return c + self.ctime[action]
    
    # implement only if you want to experiment with informed strategies
    def h(self, node):
        """Returns the heuristic applied to the Node node""" 
        toCross=node.state[0]
        result = 0
        if len(toCross)>0:
            result += self.ctime[toCross[0]] 
        if len(toCross)>2:
            result += self.ctime[toCross[2]]
        return result

#####################
# Test
initialState=(("Bono", "Larry", "Edge", "Adam"), (), 1)
goalState=((),("Bono", "Edge", "Adam", "Larry"),0)
problemInstance = U2(initialState, goalState)
soln = astar_search(problemInstance)

# to print the sequence of actions
path = path_actions(soln)
print(path)
print("-------")
# to print the sequence of states
path = path_states(soln)
print(path)
print()

# to apply all the strategies, and print some report about used resources
# NOTICE: depth_first_tree_search is commented out... guess why?
report([
    breadth_first_tree_search,
    breadth_first_graph_search,
    # depth_first_tree_search,
    depth_first_graph_search,
    astar_search
    ],
    [problemInstance])

[('Edge', 'Bono'), 'Bono', ('Larry', 'Adam'), 'Edge', ('Edge', 'Bono')]
-------
[(('Larry', 'Adam', 'Edge', 'Bono'), (), 1), (('Larry', 'Adam'), ('Edge', 'Bono'), 0), (('Larry', 'Adam', 'Bono'), ('Edge',), 1), (('Bono',), ('Larry', 'Adam', 'Edge'), 0), (('Edge', 'Bono'), ('Larry', 'Adam'), 1), ((), ('Larry', 'Adam', 'Edge', 'Bono'), 0)]

breadth_first_tree_search:
    4,857 inserted nodes |      940 tested goals |   50 cost |     944 expanded_nodes | U2
    4,857 inserted nodes |      940 tested goals |   50 cost |     944 expanded_nodes | TOTAL

breadth_first_graph_search:
       83 inserted nodes |       26 tested goals |   50 cost |      25 expanded_nodes | U2
       83 inserted nodes |       26 tested goals |   50 cost |      25 expanded_nodes | TOTAL

depth_first_graph_search:
       32 inserted nodes |       10 tested goals |   19 cost |      14 expanded_nodes | U2
       32 inserted nodes |       10 tested goals |   19 cost |      14 expanded_nodes | TOTAL

astar_search:
       

## 📊 Analisi del Report

### **breadth_first_tree_search**  
- Molto inefficiente in termini di spazio e tempo.
- Esplora un numero elevato di nodi senza tenere traccia dei nodi già visitati, il che porta a un'esplorazione ridondante e un numero elevato di espansioni.
- Il costo totale di questa strategia è 50, che è significativamente superiore al limite massimo consentito di 17 minuti.

### **breadth_first_graph_search**  
- Molto più efficiente rispetto al tree search grazie al controllo sui nodi già visitati.
- Riduce il numero di nodi espansi e inseriti, ma il costo rimane ugualmente alto, pari a 50, ben oltre il limite.
- Rispetto al tree search, è un miglioramento in termini di prestazioni, ma non raggiunge il limite di costo.

### **depth_first_graph_search**  
- Buona efficienza, migliore rispetto alla ricerca in ampiezza con grafo.
- Trova una soluzione con costo corretto di 19, che è ancora sopra il limite massimo consentito di 17 minuti.
- Non è ottimale rispetto al costo, ma ha trovato una soluzione valida in termini di espansioni.

### **astar_search**  
- Utilizza una funzione euristica che, pur essendo semplice, sembra abbastanza efficace.
- Il costo totale di 17 è esattamente il limite massimo, il che significa che questa strategia ha trovato una soluzione soddisfacente dal punto di vista del costo.
- L'efficienza è comparabile a quella della ricerca in ampiezza con grafo, suggerendo che l'euristica è stata utile ma non estremamente informativa. Nonostante ciò, è l'unica strategia che ha rispettato il limite di costo.
